In [1]:
# IMPORTAMOS LIBRERIAS
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import mysql.connector

pd.set_option('display.max_columns', None) 

In [2]:
# DEFINIR LA ESTRUCTURA DE LA BBDD
# Crear la base de datos
db = mysql.connector.connect(
    user="root",
    password="AlumnaAdalab",
    host="127.0.0.1",
    auth_plugin='mysql_native_password'
)

cursor = db.cursor()
query_create_bd = "CREATE SCHEMA IF NOT EXISTS `ABCCorporation`"

cursor.execute(query_create_bd)

In [3]:
# Leer archivo CSV en un DataFrame
df = pd.read_csv('files/df_fase1.csv')

In [4]:
# Crear la tabla 'personal_info'
cursor.execute("""
CREATE TABLE IF NOT EXISTS `abccorporation`.`personal_info` (
  `IdEmployee` INT NOT NULL,
  `EmployeeNumber` VARCHAR(45) NULL DEFAULT NULL,
  `Age` INT NULL DEFAULT NULL,
  `DateBirth` INT NULL DEFAULT NULL,
  `Gender` VARCHAR(45) NULL DEFAULT NULL,
  `MaritalStatus` VARCHAR(45) NULL DEFAULT NULL,
  `DistanceFromHome` INT NULL DEFAULT NULL,
  `Education` INT NULL DEFAULT NULL,
  `EducationField` VARCHAR(100) NULL DEFAULT NULL,
  PRIMARY KEY (`IdEmployee`)
) ENGINE = InnoDB DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_0900_ai_ci;
""")

# Crear la tabla 'employee_satisfaction'
cursor.execute("""
CREATE TABLE IF NOT EXISTS `abccorporation`.`employee_satisfaction` (
  `IdEmployeeSatisfaction` INT NOT NULL AUTO_INCREMENT,
  `EnvironmentSatisfaction` INT NULL DEFAULT NULL,
  `JobInvolvement` INT NULL DEFAULT NULL,
  `JobSatisfaction` INT NULL DEFAULT NULL,
  `RelationshipSatisfaction` INT NULL DEFAULT NULL,
  `WorkLifeBalance` INT NULL DEFAULT NULL,
  `personal_info_IdEmployee` INT NOT NULL,
  PRIMARY KEY (`IdEmployeeSatisfaction`, `personal_info_IdEmployee`),
  INDEX `fk_employee_satisfaction_personal_info_idx` (`personal_info_IdEmployee` ASC) VISIBLE,
  CONSTRAINT `fk_employee_satisfaction_personal_info`
    FOREIGN KEY (`personal_info_IdEmployee`)
    REFERENCES `abccorporation`.`personal_info` (`IdEmployee`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION
) ENGINE = InnoDB AUTO_INCREMENT = 246 DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_0900_ai_ci;
""")

# Crear la tabla 'job_data'
cursor.execute("""
CREATE TABLE IF NOT EXISTS `abccorporation`.`job_data` (
  `IdJobData` INT NOT NULL AUTO_INCREMENT,
  `Attrition` VARCHAR(45) NULL DEFAULT NULL,
  `JobLevel` INT NULL DEFAULT NULL,
  `JobRole` VARCHAR(100) NULL DEFAULT NULL,
  `NumCompaniesWorked` INT NULL DEFAULT NULL,
  `TotalWorkingYears` VARCHAR(45) NULL DEFAULT NULL,
  `YearsAtCompany` INT NULL DEFAULT NULL,
  `YearsSinceLastPromotion` INT NULL DEFAULT NULL,
  `YearsWithCurrManager` INT NULL DEFAULT NULL,
  `personal_info_IdEmployee` INT NOT NULL,
  PRIMARY KEY (`IdJobData`, `personal_info_IdEmployee`),
  INDEX `fk_job_data_personal_info1_idx` (`personal_info_IdEmployee` ASC) VISIBLE,
  CONSTRAINT `fk_job_data_personal_info1`
    FOREIGN KEY (`personal_info_IdEmployee`)
    REFERENCES `abccorporation`.`personal_info` (`IdEmployee`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION
) ENGINE = InnoDB AUTO_INCREMENT = 1355 DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_0900_ai_ci;
""")

# Crear la tabla 'job_extras'
cursor.execute("""
CREATE TABLE IF NOT EXISTS `abccorporation`.`job_extras` (
  `IdJobExtras` INT NOT NULL AUTO_INCREMENT,
  `OverTime` VARCHAR(45) NULL DEFAULT NULL,
  `BusinessTravel` VARCHAR(45) NULL DEFAULT NULL,
  `StockOptionLevel` INT NULL DEFAULT NULL,
  `TrainingTimesLastYear` INT NULL DEFAULT NULL,
  `RemoteWork` TINYINT NULL DEFAULT NULL,
  `PerformanceRating` INT NULL DEFAULT NULL,
  `personal_info_IdEmployee` INT NOT NULL,
  PRIMARY KEY (`IdJobExtras`, `personal_info_IdEmployee`),
  INDEX `fk_job_extras_personal_info1_idx` (`personal_info_IdEmployee` ASC) VISIBLE,
  CONSTRAINT `fk_job_extras_personal_info1`
    FOREIGN KEY (`personal_info_IdEmployee`)
    REFERENCES `abccorporation`.`personal_info` (`IdEmployee`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION
) ENGINE = InnoDB AUTO_INCREMENT = 460 DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_0900_ai_ci;
""")

# Crear la tabla 'salary'
cursor.execute("""
CREATE TABLE IF NOT EXISTS `abccorporation`.`salary` (
  `EmployeeSalary` INT NOT NULL AUTO_INCREMENT,
  `HourlyRate` FLOAT NULL DEFAULT NULL,
  `DailyRate` FLOAT NULL DEFAULT NULL,
  `MonthlyRate` FLOAT NULL DEFAULT NULL,
  `PercentSalaryHike` INT NULL DEFAULT NULL,
  `personal_info_IdEmployee` INT NOT NULL,
  PRIMARY KEY (`EmployeeSalary`, `personal_info_IdEmployee`),
  INDEX `fk_salary_personal_info1_idx` (`personal_info_IdEmployee` ASC) VISIBLE,
  CONSTRAINT `fk_salary_personal_info1`
    FOREIGN KEY (`personal_info_IdEmployee`)
    REFERENCES `abccorporation`.`personal_info` (`IdEmployee`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION
) ENGINE = InnoDB AUTO_INCREMENT = 1472 DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_0900_ai_ci;
""")

# Confirmar cambios
db.commit()

# Cerrar cursor y conexión
cursor.close()
db.close()

In [5]:
# Funcion para convertir a entero
def convertir(x): 
    
    # Convertir solo las columnas de tipo int64
    lista = []
 
    for tupla in x:
        lista_intermedia = []
    
        for elem in tupla:
            try:
                lista_intermedia.append(int(elem))
            except:
                lista_intermedia.append(elem)
        
        lista.append(tuple(lista_intermedia))
        
    return lista 

In [6]:
# Insertar datos

cnx = mysql.connector.connect(
    user="root",
    password="AlumnaAdalab",
    host="127.0.0.1",
    auth_plugin='mysql_native_password'
)

cursor = cnx.cursor()

### INSERCION PERSONAL_INFO

query_insertar_tablapersonal = """ INSERT INTO `ABCCorporation`.`PERSONAL_INFO` (`IdEmployee`, `EmployeeNumber`, `Age`, `DateBirth`, `Gender`, `MaritalStatus`, `DistanceFromHome`, `Education`, `EducationField`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""

datos_tabla_personal = list(set(zip
                                (df["IdEmployee"].values, 
                                 df["EmployeeNumber"].values,
                                 df["Age"].values,
                                 df["DateBirth"].values,
                                 df["Gender"].values,
                                 df["MaritalStatus"].values,
                                 df["DistanceFromHome"].values,
                                 df["Education"].values,
                                 df["EducationField"].values,
                                 )))

datos_personal = convertir(datos_tabla_personal)

id_employee = cursor.lastrowid

cursor.executemany(query_insertar_tablapersonal, datos_personal)

cnx.commit()


In [7]:
id_employee = cursor.lastrowid

In [8]:
# Tabla EMPLOYEE_SATISFACTION

#query_insertar_satisfaction = """ INSERT INTO `ABCCorporation`.`EMPLOYEE_SATISFACTION` (`EnvironmentSatisfaction`, `JobInvolvement`, `JobSatisfaction`, `RelationshipSatisfaction`, `WorkLifeBalance`, `IdEmployee`) VALUES (%s, %s, %s, %s, %s, %s)"""


query_employee_satisfaction = """
    INSERT INTO `abccorporation`.`employee_satisfaction` 
    (`EnvironmentSatisfaction`, `JobInvolvement`, `JobSatisfaction`, `RelationshipSatisfaction`, `WorkLifeBalance`, `personal_info_IdEmployee`)
    VALUES (%s, %s, %s, %s, %s, %s)
    """

datos_tabla_satisfaction = list(set(zip
                                (df["EnvironmentSatisfaction"].values,
                                 df["JobInvolvement"].values,
                                 df["JobSatisfaction"].values,
                                 df["RelationshipSatisfaction"].values,
                                 df["WorkLifeBalance"].values,
                                 df["IdEmployee"].values
                                 )))

datos_satisfaction = convertir(datos_tabla_satisfaction)

id_employee = cursor.lastrowid

cursor.executemany(query_employee_satisfaction, datos_satisfaction)

satisfaction_id = cursor.lastrowid

cnx.commit()

In [9]:
# Tabla JOB_DATA

# query_insertar_job_data = """ INSERT INTO `ABCCorporation`.`JOB_DATA` (`Attrition`, `JobLevel`, `JobRole`, `NumCompaniesWorked`, `TotalWorkingYears`, `YearsAtCompany`,`YearsSinceLastPromotion`, `YearsWithCurrManager`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""



query_job_data = """
    INSERT INTO `abccorporation`.`job_data`
    (`Attrition`, `JobLevel`, `JobRole`, `NumCompaniesWorked`, `TotalWorkingYears`, 
    `YearsAtCompany`, `YearsSinceLastPromotion`, `YearsWithCurrManager`, `personal_info_IdEmployee`)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

datos_tabla_job_data = list(set(zip
                                (df["Attrition"].values,
                                 df["JobLevel"].values,
                                 df["JobRole"].values,
                                 df["NumCompaniesWorked"].values,
                                 df["TotalWorkingYears"].values,
                                 df["YearsAtCompany"].values,
                                 df["YearsSinceLastPromotion"].values,
                                 df["YearsWithCurrManager"].values,
                                 df["IdEmployee"].values
                                 )))

datos_job_data = convertir(datos_tabla_job_data)


cursor.executemany(query_job_data, datos_job_data)

job_data_id = cursor.lastrowid

cnx.commit()

In [10]:
def extraer_datos(x):
    
    lista = []
    
    for dato in x:
        lista.append(cursor.lastrowid)
    
    return lista

In [11]:
# Tabla SALARY

# query_insertar_salary = """ INSERT INTO `ABCCorporation`.`SALARY` (`HourlyRate`, `DailyRate`, `MonthlyRate`, `PercentSalaryHike`) VALUES (%s, %s, %s, %s)"""

query_salary = """
    INSERT INTO `abccorporation`.`salary`
    (`HourlyRate`, `DailyRate`, `MonthlyRate`, `PercentSalaryHike`, `personal_info_IdEmployee`)
    VALUES (%s, %s, %s, %s, %s)
    """


datos_tabla_salary = list(set(zip
                                (df["HourlyRate"].values,
                                 df["DailyRate"].values,
                                 df["MonthlyRate"].values,
                                 df["PercentSalaryHike"].values,
                                 df["IdEmployee"].values
                                 )))

datos_salary = convertir(datos_tabla_salary)
salary_id = cursor.lastrowid  # Obtener el IdSalary generado

cursor.executemany(query_salary, datos_salary)


cnx.commit()

In [12]:
# Tabla JOB_EXTRAS

# query_insertar_job_extras = """ INSERT INTO `ABCCorporation`.`JOB_EXTRAS` (`OverTime`, `BusinessTravel`, `StockOptionLevel`, `TrainingTimesLastYear`, `RemoteWork`, `PerformanceRating`) VALUES (%s, %s, %s, %s, %s, %s)"""

query_job_extras = """
    INSERT INTO `abccorporation`.`job_extras`
    (`OverTime`, `BusinessTravel`, `StockOptionLevel`, `TrainingTimesLastYear`, 
    `RemoteWork`, `PerformanceRating`, `personal_info_IdEmployee`)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """ 

datos_tabla_job_extras = list(set(zip
                                (df["OverTime"].values,
                                 df["BusinessTravel"].values,
                                 df["StockOptionLevel"].values,
                                 df["TrainingTimesLastYear"].values,
                                 df["RemoteWork"].values,
                                 df["PerformanceRating"].values,
                                 df["IdEmployee"].values
                                 )))

datos_job_extras = convertir(datos_tabla_job_extras)
job_extras_id = cursor.lastrowid  # Obtener el IdJobExtras generado

cursor.executemany(query_job_extras, datos_job_extras)


cnx.commit()